In [258]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from itertools import chain


In [259]:
class Web:
    def __init__(self, url):
        self.url = url
    
    def get_html(self) -> 'html': 
        return requests.get(self.url).text
    

In [260]:
data = pd.DataFrame(columns=['park_name', 'description', 'position', 'area', 'attractions'])


In [262]:
def format_url(park):
    park = re.sub(r'\W', '-', park)
    return re.sub(r'--+', '-', park)


In [263]:
with open(r'D:\WarsawParkInfo\data\park', mode='r', encoding='utf-8') as f:
    parks_name = f.read().split(',\n')
parks_name = [format_url(park) for park in parks_name]


In [264]:
pattern = 'https://eko.um.warszawa.pl/-/{}?redirect=%2Fparki_'
pages = [Web(pattern.format(park)).get_html() for park in parks_name]


In [265]:
pages_html_format = {name:BeautifulSoup(page, 'html.parser').find_all('div', attrs={'class':'article-description'}) for name, page in zip(parks_name,pages)}


In [266]:
# ['park_name', 'description', 'position', 'area', 'attractions'])
poor_parks = list()
table = {
'park_name': list(),
'attractions': list(),
'area': list(),
'position': list(),
'description': list()}

for park in pages_html_format:
    content = list()
    p_tags = pages_html_format[park][0].find_all('p')
    p_tags.reverse()
    for p in p_tags:
        content.append(p.text)
    try:
        table['park_name'].append(park)
        table['attractions'].append(content[0])
        table['area'].append(content[1])
        table['position'].append(content[2])
        table['description'].append(list(chain(content[-1:2:-1])))
    except IndexError:
        poor_parks.append(park)

In [269]:
# pd.DataFrame(table).to_excel(r'D:\WarsawParkInfo\data\table.xlsx')